In [570]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

In [571]:
data = pd.read_csv('test_data.csv')

Убираем из данных реплики клиента, так как они не нужны для анализа слов оператора

In [572]:
data = data[data.role == 'manager']

Создаем структуру данных, в которую будет записываться информация о разговоре с клиентом

In [573]:
analyze = {i: {'greetings': None,
               'greetings_line': None,
               'name': None,
               'name_line': None,
               'company': None,
               'company_line': None,
               'farewell': None,
               'farewell_line': None,
               'greetings_and_farewell': None
               } for i in data.dlg_id.unique()}

Поиск нужных слов будет происходить с помощью регулярных выражений, так как перевод разговора в текст произошел без глобальных  ошибок, следовательно не нужно проводить лемматизацию текста

In [574]:
names_score = 0.5
morph = MorphAnalyzer()
greetings = 'привет|здравствуйте|доброе утро|добрый день|добрый вечер|доброй ночи|здравствуй|добрый утро'
farewell = 'пока|до свидания|увидимся|до встречи'
name = 'зовут|имя|это'
company = 'компания'

Данная функция нужна для морфологического анализа слов и будет использоваться при поиске названия компании

In [575]:
def word_parse(word: str) -> tuple[str, int]:
    morph_parse = morph.parse(word)
    tags = ''
    for p in morph_parse:
        tags += str(p.tag) + ','
    return tags, len(morph_parse)

При поиске каждого пункта для анализа сначала проверяем налиличие уже заполненных данных в каждом поле, далее ищем ключевые слова с помощью регулярных выражений

In [576]:
def text_analyze(dlg_id: int, line: int, text: str, analyze: dict) -> None:

    lower_text = text.lower() # переводим текст в нижний регистр
    tokenize_text = word_tokenize(lower_text) # токенизируем текст для последущего использования в поиске названия компании
    len_line = len(tokenize_text)

    if analyze[dlg_id]['greetings'] is None:
        if re.findall(greetings, lower_text):
            analyze[dlg_id]['greetings'] = re.findall(greetings, lower_text)[0]
            analyze[dlg_id]['greetings_line'] = line

    if analyze[dlg_id]['farewell'] is None:
        if re.findall(farewell, lower_text):
            analyze[dlg_id]['farewell'] = re.findall(farewell, lower_text)[0]
            analyze[dlg_id]['farewell_line'] = line

    # для поиска имени и названия компании необходимо найти ключевое слово
    # от ключего слова отступаем в разные стороны, пока не найдем имя
    # для поиска названия компании отступаем только в одну сторону
    # поиск имени и компании осуществляется с помощью морфологического анализатора
    if analyze[dlg_id]['name'] is None:
        if re.findall(name, lower_text):
            not_find_name = True
            for word in tokenize_text:
                if word in name and not_find_name:
                    start = tokenize_text.index(word)
                    count_of_words = 1

                    if start - count_of_words >= 0 or start + count_of_words < len_line:
                        for index in (start - count_of_words, start + count_of_words):
                            for parse in morph.parse(tokenize_text[index]):
                                if 'Name' in parse.tag and parse.score >= names_score:
                                    analyze[dlg_id]['name'] = parse.word
                                    analyze[dlg_id]['name_line'] = line
                                    not_find_name = False
                        count_of_words += 1

    if analyze[dlg_id]['company'] is None:
        if company in tokenize_text:

            start = tokenize_text.index(company) + 1
            company_name = ''
            word = ''
            company_is_valid = True

            while company_is_valid:
                company_name += word if company_name == '' else ' ' + word
                word = tokenize_text[start]
                tags, len_of_morph_parse = word_parse(word)
                if not('NOUN' in tags or 'ADJF' in tags or len_of_morph_parse > 3):
                    company_is_valid = False
                start += 1

            if company_name != '':
                analyze[dlg_id]['company'] = company_name
                analyze[dlg_id]['company_line'] = line

В цикле вызываем процедуру заполнения структуры для анализа

In [577]:
for row in data.itertuples():
    text_analyze(row.dlg_id, row.line_n, row.text, analyze)

Заполняем галочку, что менеджер поздоровался и попрощался

In [578]:
for i in analyze.values():
    i['greetings_and_farewell'] = not(i['greetings'] is None or i['farewell'] is None)

Ответ на задание

In [579]:
analyze

{0: {'greetings': 'здравствуйте',
  'greetings_line': 1,
  'name': 'ангелина',
  'name_line': 3,
  'company': 'диджитал бизнес',
  'company_line': 3,
  'farewell': 'до свидания',
  'farewell_line': 108,
  'greetings_and_farewell': True},
 1: {'greetings': 'здравствуйте',
  'greetings_line': 1,
  'name': 'ангелина',
  'name_line': 2,
  'company': 'диджитал бизнес',
  'company_line': 2,
  'farewell': 'до свидания',
  'farewell_line': 54,
  'greetings_and_farewell': True},
 2: {'greetings': 'здравствуйте',
  'greetings_line': 2,
  'name': 'ангелина',
  'name_line': 3,
  'company': 'диджитал бизнес',
  'company_line': 3,
  'farewell': None,
  'farewell_line': None,
  'greetings_and_farewell': False},
 3: {'greetings': 'добрый день',
  'greetings_line': 1,
  'name': 'максим',
  'name_line': 2,
  'company': 'китобизнес',
  'company_line': 2,
  'farewell': None,
  'farewell_line': None,
  'greetings_and_farewell': False},
 4: {'greetings': None,
  'greetings_line': None,
  'name': None,
  'na